<a href="https://colab.research.google.com/github/Taha-Sayyed/Machine-Learning-Project/blob/main/Classifier_model_for_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **This Notebook is for Testing purpose**

**This Notebook do not contain code for Model Training. We are importing the model & embedding in the form of .pkl file and .npz file**

In [ ]:
model_path="/content/drive/MyDrive/Face_Recognition/svm_model_updated.pkl"

In [ ]:
import pickle

# Open and load the model from Drive
with open(model_path, 'rb') as f:
    model = pickle.load(f)

print("Model loaded successfully!")


Model loaded successfully!


In [ ]:
model

SVC(kernel='linear', probability=True)

In [ ]:
import cv2 as cv
import numpy as np
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' #This variable determines the minimum severity of log messages that TensorFlow’s C++ backend will display.

In [ ]:
!pip install mtcnn

In [ ]:
!pip install keras-facenet

In [ ]:
from keras_facenet import FaceNet
embedder = FaceNet()

def get_embedding(face_img):
    face_img = face_img.astype('float32') # 3D(160x160x3)
    face_img = np.expand_dims(face_img, axis=0)
    # 4D (Nonex160x160x3)
    yhat= embedder.embeddings(face_img)
    return yhat[0] # 512D image (1x1x512)

In [ ]:
# Load stored X and Y if saved earlier
data = np.load('/content/drive/MyDrive/Face_Recognition/face_embeddig_for_3_class.npz')

X = data['arr_0']  # Face embeddings (features)
Y = data['arr_1']  # Corresponding labels (classes)

print("Loaded X shape:", X.shape)
print("Loaded Y shape:", Y.shape)


Loaded X shape: (732, 512)
Loaded Y shape: (732,)


In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
encoder.fit(Y)
Y=encoder.transform(Y)

# **Testing**

In [ ]:
#For Testing
from mtcnn.mtcnn import MTCNN

detector = MTCNN()


t_im = cv.imread("/content/drive/MyDrive/Face_Recognition/test_4.jpeg")
t_im_rgb = cv.cvtColor(t_im, cv.COLOR_BGR2RGB)


faces = detector.detect_faces(t_im_rgb)
print("Number of faces detected:", len(faces))

# List to store embeddings for all detected faces
test_embeddings = []
for i in faces:
  print(f"The confidence score for the detected face are {i['confidence']}")
print("\n")

# Loop over each detected face
for face in faces:

    if face['confidence']>0.95:


      x, y, w, h = face['box']
      x, y = abs(x), abs(y)  # Ensure coordinates are positive


      cv.rectangle(t_im_rgb, (x, y), (x + w, y + h), (0, 255, 0), 2)


      face_crop = t_im_rgb[y:y + h, x:x + w]


      face_resized = cv.resize(face_crop, (160, 160))
      print(f"The confidence score of the given below image is {face['confidence']}")
      cv2_imshow(face_resized)


      embedding = get_embedding(face_resized)

      # Append the embedding to the list
      # Now, test_embeddings is a list containing the embeddings for each detected face.
      test_embeddings.append(embedding)

      #Logic
      threshold=0.70
      final_predictions = []
      predictions_proba = model.predict_proba(test_embeddings)
      predictions = model.predict(test_embeddings)
      for i, probs in enumerate(predictions_proba):
        max_prob = np.max(probs)  # Get the highest probability for this sample
        if max_prob < threshold:
          final_predictions.append("Unknown")
        else:
          final_predictions.append(predictions[i])
      #Logic

plt.figure(figsize=(10, 8))
plt.imshow(t_im_rgb)
plt.axis('off')
plt.show()



In [ ]:

def inverse_transform(final_predictions):
  Y_pred_name = []
  for pred in final_predictions:
      if pred == "Unknown":
          Y_pred_name.append("Unknown")
      else:
          Y_pred_name.append(encoder.inverse_transform([pred])[0])

  print(Y_pred_name)

inverse_transform(final_predictions)

['Unknown', 'taha_sayyed']


In [ ]:
print(predictions_proba)
print(predictions_proba.shape)

[[0.68642527 0.30275776 0.01081697]
 [0.02358387 0.96821378 0.00820235]]
(2, 3)
